---



# Unit 2 - Part 2a: The Anatomy of a Prompt

## 1. Introduction: Stochasticity (Randomness)

Why does the AI give different answers? Because it is **Stochastic** (Random).

It predicts the NEXT TOKEN based on probability.

### Visualizing the Prediction
Input: `"The sky is..."`

| Word | Probability | Selected? (Temp=0) | Selected? (Temp=1) |
|------|-------------|--------------------|--------------------|
| Blue | 80% | ✅ | ❌ |
| Gray | 15% | ❌ | ✅ |
| Green| 1% | ❌ | ❌ |

Prompt Engineering is the art of **manipulating these probabilities**.

In [1]:
# %pip install python-dotenv --upgrade --quiet langchain langchain-google-genai
!pip install --upgrade --quiet langchain langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.5/500.5 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.1/158.1 kB 13.2 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
userdata.get('google_api')

In [3]:
# Setup
# from dotenv import load_dotenv
# load_dotenv()

import getpass
import os
from langchain_google_genai import ChatGoogleGenerativeAI

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API Key: ")

# Using Low Temp for consistent comparison
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.0)

Enter your Google API Key: ··········


## 2. The CO-STAR Framework (simplified)

A good prompt usually has:
1.  **C**ontext (Who are you? Who acts?)
2.  **O**bjective (What is the task?)
3.  **S**tyle (Formal? Funny?)
4.  **T**one (Empathetic? Direct?)
5.  **A**udience (Who is reading this?)
6.  **R**esponse Format (JSON? List?)

Let's compare a **Lazy Prompt** vs a **CO-STAR Prompt**.

In [4]:
# The Task: Reject a candidate for a job.
task = "Write a rejection email to a candidate."

print("--- LAZY PROMPT ---")
print(llm.invoke(task).content)

--- LAZY PROMPT ---
Here are a few options for a rejection email, ranging from a standard template to one for a candidate who interviewed. Choose the one that best fits your situation.

---

**Option 1: Standard Rejection (No Interview)**

This is suitable for candidates who applied but were not selected for an interview.

**Subject: Update on Your Application for [Job Title] at [Company Name]**

Dear [Candidate Name],

Thank you for your interest in the [Job Title] position at [Company Name] and for taking the time to submit your application.

We received a large number of highly qualified applications for this role. While your qualifications are impressive, we have decided to move forward with other candidates whose profiles were a closer match for the specific requirements of this position at this time.

We appreciate you considering [Company Name] as a potential employer and wish you the best of luck in your job search and future endeavors.

Sincerely,

[Your Name]
[Your Title]
[Co

## 3. Hallucination vs. Creativity

Did the model make up a reason?
Since we didn't give it facts, it **Predicted the most likely reason** (Usually "Experience" or "Volume of applications").

**This is NOT a bug.** It is a feature. The model is *completing the pattern* of a rejection email.

In [5]:
structured_prompt = """
# Context
You are an HR Manager at a quirky startup called 'RocketBoots'.

# Objective
Write a rejection email to a candidate named Bob.

# Constraints
1. Be extremely brief (under 50 words).
2. Do NOT say 'we found someone better'. Say 'the role changed'.
3. Sign off with 'Keep flying'.

# Output Format
Plain text, no subject line.
"""

print("--- STRUCTURED PROMPT ---")
print(llm.invoke(structured_prompt).content)

--- STRUCTURED PROMPT ---
Hi Bob,

Thank you for your interest in RocketBoots. We appreciate your time and effort.

While your application was impressive, the requirements for this role have recently changed. We won't be moving forward with your candidacy at this time.

Keep flying,
RocketBoots HR


## 4. Key Takeaway: Ambiguity is the Enemy

Every piece of information you leave out is a gap the model MUST fill with probability.
- If you don't say "Be brief", it picks the most probable length (Avg email length).
- If you don't say "Be rude", it picks the most probable tone (Polite/Neutral).

## Assignment

Write a structured prompt to generate a **Python Function**.
- **Context:** You are a Senior Python Dev.
- **Objective:** Write a function to reverse a string.
- **Constraint:** It must use recursion (no slicing `[::-1]`).
- **Style:** Include detailed docstrings.

In [6]:
structured_prompt = """
# Context
You are a Senior Python Dev.

# Objective
Write a function to reverse a string.

# Constraint
It must use recursion (no slicing [::-1]).

# Style
Include detailed docstrings.
"""

print("--- STRUCTURED PROMPT ---")
print(llm.invoke(structured_prompt).content)

--- STRUCTURED PROMPT ---
As a Senior Python Developer, I'll craft a recursive string reversal function that adheres to the specified constraints and includes comprehensive documentation.

```python
import sys

def reverse_string_recursive(s: str) -> str:
    """
    Reverses a given string using a recursive approach.

    This function takes an input string and returns a new string with its
    characters in reverse order. It strictly adheres to the constraint of
    using recursion and explicitly avoids direct string slicing for reversal
    (e.g., `s[::-1]`).

    The recursive logic is as follows:
    1.  **Base Case**: If the input string `s` is empty or contains only
        a single character, it is already considered "reversed". In this
        scenario, the function returns the string `s` itself. This is the
        critical stopping condition that prevents infinite recursion.

    2.  **Recursive Step**: For any string `s` with two or more characters:
        a.  It takes the

---



# Unit 2 - Part 2b: Zero-Shot to Few-Shot

## 1. Introduction: In-Context Learning

How does the model learn without training?
This is called **In-Context Learning**.

### The Attention Mechanism (Flowchart)
When you ask a question, the model "looks back" at the previous text to find patterns.

```mermaid
graph TD
    Input[Current Input: 'Angry + Hungry'] -->|Attention Query| History
    subgraph History [The Prompt Examples]
        Ex1[Ex1: Breakfast + Lunch = Brunch]
        Ex2[Ex2: Chill + Relax = Chillax]
    end
    History -->|Pattern Found: Mix words & define| Prediction[Output: Hangry]
```

In [7]:
# Setup
# from dotenv import load_dotenv
# load_dotenv()

# import getpass
# import os
# from langchain_google_genai import ChatGoogleGenerativeAI

# if "GOOGLE_API_KEY" not in os.environ:
#     os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API Key: ")

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.5)

## 2. Zero-Shot (No Context)

The model relies purely on its training data.

In [8]:
prompt_zero = "Combine 'Angry' and 'Hungry' into a funny new word."
print(f"Zero-Shot: {llm.invoke(prompt_zero).content}")

Zero-Shot: The most common and widely accepted (and hilarious) word for this is:

**Hangry**


## 3. Few-Shot (Pattern Matching)

We provide examples. The Attention Mechanism attends to the **Structure** (`Input -> Output`) and the **Tone** (Sarcasm).

In [9]:
prompt_few = """
Combine words into a funny new word. Give a sarcastic definition.

Input: Breakfast + Lunch
Output: Brunch (An excuse to drink alcohol before noon)

Input: Chill + Relax
Output: Chillax (What annoying people say when you have panic attacks)

Input: Angry + Hungry
Output:
"""
print(f"Few-Shot: {llm.invoke(prompt_few).content}")

Few-Shot: Output: Hangry (A completely legitimate medical condition that excuses all bad behavior until food is presented.)


## 4. Critical Analysis

If you provide **bad examples**, the model will learn the **bad pattern**.
This is why Data Quality in your prompt is just as important as code quality.

---



# Unit 2 - Part 2c: Advanced Templates & Theory

## 1. Theory: Engineering vs. Training

### Hard Prompts (Prompt Engineering)
- **What:** You change the text input.
- **Cost:** Cheap, fast, easy to iterate.
- **Use Case:** Prototyping, General tasks.

### Soft Prompts (Fine Tuning)
- **What:** You change the model's internal weights (mathematically).
- **Cost:** Expensive, slow, needs data.
- **Use Case:** Domain specificity (Medical, Legal), Behavioral change.

In [10]:
# Setup
# from dotenv import load_dotenv
# load_dotenv()

# import getpass
# import os
# from langchain_google_genai import ChatGoogleGenerativeAI

# if "GOOGLE_API_KEY" not in os.environ:
#     os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API Key: ")
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

## 2. Dynamic Few-Shotting

If you have 1000 examples, you can't fit them all in the context window.
We use a **Selector** to pick the best ones.

### The Selector Flow (Flowchart)
```mermaid
graph LR
    Input[User Input] -->|Semantic Search| Database[Example Database]
    Database -->|Top 3 Matches| Selector
    Selector -->|Inject| Prompt
```

In [11]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

# 1. Our Database of Examples
examples = [
    {"input": "The internet is down.", "output": "We are observing connectivity latency."},
    {"input": "This code implies a bug.", "output": "The logic suggests unintended behavior."},
    {"input": "I hate this feature.", "output": "This feature does not align with my preferences."},
]

# 2. Template for ONE example
example_fmt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}")
])

# 3. The Few-Shot Container
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_fmt,
    examples=examples
)

# 4. The Final Chain
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a Corpo-Speak Translator. Rewrite the input to sound professional."),
    few_shot_prompt,      # Inject examples here
    ("human", "{text}")
])

chain = final_prompt | llm

print(chain.invoke({"text": "This app sucks."}).content)

The application's effectiveness is currently suboptimal.


## 3. Analysis

Using `FewShotChatMessagePromptTemplate` creates a clean separation between instructions and data. This helps the Attention Mechanism focus on the right things.